# Integrals and Multivariate Calculus
## From Basics to Machine Learning Applications

---

## Table of Contents
1. [Integration Fundamentals](#integration)
2. [Numerical Integration](#numerical)
3. [Multiple Integrals](#multiple)
4. [Probability Applications](#probability)
5. [Expected Value and Moments](#expectation)
6. [Multivariate Calculus](#multivariate)
7. [Visualizations](#visualizations)
8. [Practice Problems](#practice)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy import integrate
from scipy import stats

np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')

---
# 1. INTEGRATION FUNDAMENTALS

## 1.1 What is Integration?

Integration is the inverse of differentiation. If differentiation finds the rate of change, integration finds the **accumulated total**.

The **definite integral** of f(x) from a to b:

$$\int_a^b f(x) \, dx = F(b) - F(a)$$

where F(x) is the **antiderivative** of f(x).

### Geometric Interpretation
The definite integral represents the **signed area** under the curve.

In [ ]:
# Example: Visualize area under curve
def f(x):
    return x**2

x = np.linspace(0, 3, 100)
y = f(x)

fig, ax = plt.subplots(figsize=(10, 6))

# Plot function
ax.plot(x, y, 'b-', linewidth=2, label='$f(x) = x^2$')

# Fill area under curve
x_fill = np.linspace(0, 2, 100)
y_fill = f(x_fill)
ax.fill_between(x_fill, y_fill, alpha=0.3, color='blue',
                label=f'$\\int_0^2 x^2 dx = {2**3/3:.3f}$')

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Definite Integral as Area Under Curve')
ax.legend()
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='k', linewidth=0.5)
ax.axvline(x=0, color='k', linewidth=0.5)

plt.show()

# Analytical: ∫x² dx = x³/3
# From 0 to 2: 2³/3 - 0³/3 = 8/3
print(f"Analytical result: ∫₀² x² dx = 8/3 = {8/3:.4f}")

## 1.2 Common Integration Rules

### Power Rule
$$\int x^n \, dx = \frac{x^{n+1}}{n+1} + C \quad (n \neq -1)$$

### Exponential
$$\int e^x \, dx = e^x + C$$

### Trigonometric
$$\int \sin(x) \, dx = -\cos(x) + C$$
$$\int \cos(x) \, dx = \sin(x) + C$$

In [ ]:
# Verify with numerical integration
from scipy.integrate import quad

# Example 1: ∫₀¹ x³ dx = 1/4
result, error = quad(lambda x: x**3, 0, 1)
print(f"∫₀¹ x³ dx = {result:.6f} (analytical: 0.25)")

# Example 2: ∫₀^π sin(x) dx = 2
result, error = quad(np.sin, 0, np.pi)
print(f"∫₀^π sin(x) dx = {result:.6f} (analytical: 2.0)")

# Example 3: ∫₀¹ e^x dx = e - 1
result, error = quad(np.exp, 0, 1)
print(f"∫₀¹ eˣ dx = {result:.6f} (analytical: {np.e - 1:.6f})")

# Example 4: ∫₋∞^∞ e^(-x²) dx = √π (Gaussian integral)
result, error = quad(lambda x: np.exp(-x**2), -np.inf, np.inf)
print(f"∫₋∞^∞ e^(-x²) dx = {result:.6f} (analytical: √π = {np.sqrt(np.pi):.6f})")

## 1.3 Integration by Substitution

For $\int f(g(x)) \cdot g'(x) \, dx$, let $u = g(x)$:

$$\int f(g(x)) \cdot g'(x) \, dx = \int f(u) \, du$$

In [ ]:
# Example: ∫ 2x·e^(x²) dx
# Let u = x², then du = 2x dx
# ∫ e^u du = e^u = e^(x²)

def integrand(x):
    return 2 * x * np.exp(x**2)

def antiderivative(x):
    return np.exp(x**2)

# Verify: ∫₀¹ 2x·e^(x²) dx = e¹ - e⁰ = e - 1
result, _ = quad(integrand, 0, 1)
analytical = antiderivative(1) - antiderivative(0)

print("∫₀¹ 2x·e^(x²) dx")
print(f"Numerical: {result:.6f}")
print(f"Analytical: e - 1 = {analytical:.6f}")

---
# 2. NUMERICAL INTEGRATION

## 2.1 Riemann Sums

Approximate the integral by dividing the area into rectangles.

In [ ]:
def riemann_sum(f, a, b, n, method='midpoint'):
    """Compute Riemann sum approximation."""
    dx = (b - a) / n
    
    if method == 'left':
        x = np.linspace(a, b - dx, n)
    elif method == 'right':
        x = np.linspace(a + dx, b, n)
    elif method == 'midpoint':
        x = np.linspace(a + dx/2, b - dx/2, n)
    
    return np.sum(f(x) * dx)

# Compare methods for ∫₀² x² dx
f = lambda x: x**2
exact = 8/3

for n in [4, 10, 100, 1000]:
    left = riemann_sum(f, 0, 2, n, 'left')
    right = riemann_sum(f, 0, 2, n, 'right')
    mid = riemann_sum(f, 0, 2, n, 'midpoint')
    
    print(f"n={n:4d}: Left={left:.4f}, Right={right:.4f}, Mid={mid:.4f} (exact={exact:.4f})")

In [ ]:
# Visualize Riemann sums
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

f = lambda x: x**2
x = np.linspace(0, 2, 100)
n = 8
dx = 2/n

methods = ['left', 'right', 'midpoint']
titles = ['Left Riemann Sum', 'Right Riemann Sum', 'Midpoint Rule']

for ax, method, title in zip(axes, methods, titles):
    # Plot function
    ax.plot(x, f(x), 'b-', linewidth=2)
    
    # Draw rectangles
    for i in range(n):
        if method == 'left':
            xi = i * dx
        elif method == 'right':
            xi = (i + 1) * dx
        else:
            xi = (i + 0.5) * dx
        
        rect = plt.Rectangle((i*dx, 0), dx, f(xi), 
                             fill=True, alpha=0.3, edgecolor='black')
        ax.add_patch(rect)
    
    approx = riemann_sum(f, 0, 2, n, method)
    ax.set_title(f'{title}\nApprox = {approx:.4f}')
    ax.set_xlabel('x')
    ax.set_ylabel('y')

plt.tight_layout()
plt.show()

## 2.2 Trapezoidal and Simpson's Rules

In [ ]:
def trapezoidal(f, a, b, n):
    """Trapezoidal rule integration."""
    x = np.linspace(a, b, n+1)
    y = f(x)
    dx = (b - a) / n
    return dx * (y[0]/2 + np.sum(y[1:-1]) + y[-1]/2)

def simpsons(f, a, b, n):
    """Simpson's rule integration (n must be even)."""
    if n % 2 != 0:
        n += 1
    x = np.linspace(a, b, n+1)
    y = f(x)
    dx = (b - a) / n
    return dx/3 * (y[0] + 4*np.sum(y[1:-1:2]) + 2*np.sum(y[2:-1:2]) + y[-1])

# Compare accuracy for ∫₀^π sin(x) dx = 2
f = np.sin
exact = 2.0

print("Comparison of numerical integration methods:")
print("\nn       Trapez.    Simpson's")
print("-" * 30)
for n in [4, 10, 20, 50, 100]:
    trap = trapezoidal(f, 0, np.pi, n)
    simp = simpsons(f, 0, np.pi, n)
    print(f"{n:4d}   {trap:.6f}   {simp:.6f}")
print(f"Exact: {exact:.6f}")

## 2.3 Monte Carlo Integration

Use random sampling to estimate integrals - especially useful in high dimensions!

In [ ]:
def monte_carlo_integrate(f, a, b, n_samples):
    """Monte Carlo integration."""
    x = np.random.uniform(a, b, n_samples)
    return (b - a) * np.mean(f(x))

# Example: ∫₀² x² dx = 8/3
f = lambda x: x**2
exact = 8/3

print("Monte Carlo Integration: ∫₀² x² dx")
print("\nSamples    Estimate    Error")
print("-" * 35)

for n in [100, 1000, 10000, 100000, 1000000]:
    estimate = monte_carlo_integrate(f, 0, 2, n)
    error = abs(estimate - exact)
    print(f"{n:7d}   {estimate:.6f}   {error:.6f}")

print(f"\nExact: {exact:.6f}")

In [ ]:
# Visualize Monte Carlo convergence
np.random.seed(42)
n_max = 10000
f = lambda x: x**2
exact = 8/3

# Generate all samples at once
samples = np.random.uniform(0, 2, n_max)
values = f(samples)

# Running estimate
running_mean = np.cumsum(values) / np.arange(1, n_max + 1)
running_estimate = 2 * running_mean

plt.figure(figsize=(12, 5))
plt.plot(running_estimate, 'b-', alpha=0.7, linewidth=0.5)
plt.axhline(y=exact, color='r', linestyle='--', linewidth=2, label=f'Exact = {exact:.4f}')
plt.xlabel('Number of samples')
plt.ylabel('Estimate')
plt.title('Monte Carlo Integration Convergence')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xscale('log')
plt.show()

---
# 3. MULTIPLE INTEGRALS

## 3.1 Double Integrals

The double integral over a region R:

$$\iint_R f(x, y) \, dA = \int_a^b \int_{g(x)}^{h(x)} f(x, y) \, dy \, dx$$

In [ ]:
# Example: ∫∫ xy dA over [0,1] × [0,1]
# = ∫₀¹ ∫₀¹ xy dy dx = ∫₀¹ x[y²/2]₀¹ dx = ∫₀¹ x/2 dx = [x²/4]₀¹ = 1/4

from scipy.integrate import dblquad

# Define integrand
def f(y, x):  # Note: dblquad expects (y, x)
    return x * y

# Compute double integral
result, error = dblquad(f, 0, 1, lambda x: 0, lambda x: 1)

print("∫₀¹ ∫₀¹ xy dy dx")
print(f"Numerical: {result:.6f}")
print(f"Analytical: 1/4 = {0.25:.6f}")

In [ ]:
# Visualize the volume under surface
fig = plt.figure(figsize=(12, 5))

# 3D surface
ax1 = fig.add_subplot(121, projection='3d')
x = np.linspace(0, 1, 30)
y = np.linspace(0, 1, 30)
X, Y = np.meshgrid(x, y)
Z = X * Y

ax1.plot_surface(X, Y, Z, cmap='viridis', alpha=0.8)
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('z = xy')
ax1.set_title('Surface: z = xy')

# Contour plot
ax2 = fig.add_subplot(122)
contour = ax2.contourf(X, Y, Z, levels=20, cmap='viridis')
plt.colorbar(contour, ax=ax2, label='z = xy')
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_title('Contour Plot')

plt.tight_layout()
plt.show()

In [ ]:
# Example with non-rectangular region
# ∫∫ x²y dA over region bounded by y=0, y=x, x=1
# = ∫₀¹ ∫₀ˣ x²y dy dx = ∫₀¹ x²[y²/2]₀ˣ dx = ∫₀¹ x⁴/2 dx = [x⁵/10]₀¹ = 1/10

def f(y, x):
    return x**2 * y

result, error = dblquad(f, 0, 1, lambda x: 0, lambda x: x)

print("∫₀¹ ∫₀ˣ x²y dy dx (triangular region)")
print(f"Numerical: {result:.6f}")
print(f"Analytical: 1/10 = {0.1:.6f}")

## 3.2 Triple Integrals

In [ ]:
from scipy.integrate import tplquad

# Example: ∫∫∫ xyz dV over unit cube [0,1]³
# = [x²/2]₀¹ [y²/2]₀¹ [z²/2]₀¹ = 1/8

def f(z, y, x):
    return x * y * z

result, error = tplquad(f, 0, 1, 
                        lambda x: 0, lambda x: 1,
                        lambda x, y: 0, lambda x, y: 1)

print("∫₀¹ ∫₀¹ ∫₀¹ xyz dz dy dx")
print(f"Numerical: {result:.6f}")
print(f"Analytical: 1/8 = {0.125:.6f}")

## 3.3 Change of Variables (Jacobian)

When changing variables from (x, y) to (u, v):

$$\iint_R f(x, y) \, dx \, dy = \iint_S f(x(u,v), y(u,v)) \cdot |J| \, du \, dv$$

where $|J|$ is the absolute value of the Jacobian determinant.

In [ ]:
# Example: Convert to polar coordinates
# x = r cos(θ), y = r sin(θ)
# Jacobian |J| = r

# ∫∫ e^(-x²-y²) dA over entire plane = π
# In polar: ∫₀^∞ ∫₀^(2π) e^(-r²) r dr dθ

# Cartesian (approximate with finite bounds)
def f_cartesian(y, x):
    return np.exp(-x**2 - y**2)

result_cart, _ = dblquad(f_cartesian, -5, 5, lambda x: -5, lambda x: 5)

# Polar coordinates
def f_polar(theta, r):
    return np.exp(-r**2) * r  # Include Jacobian r

result_polar, _ = dblquad(f_polar, 0, 5, lambda r: 0, lambda r: 2*np.pi)

print("∫∫ e^(-x²-y²) dA (Gaussian integral)")
print(f"Cartesian: {result_cart:.6f}")
print(f"Polar: {result_polar:.6f}")
print(f"Exact: π = {np.pi:.6f}")

---
# 4. PROBABILITY APPLICATIONS

## 4.1 Probability Density Functions

For a continuous random variable, the PDF f(x) satisfies:

$$P(a \leq X \leq b) = \int_a^b f(x) \, dx$$

$$\int_{-\infty}^{\infty} f(x) \, dx = 1$$

In [ ]:
# Normal distribution PDF
def normal_pdf(x, mu=0, sigma=1):
    return (1 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-((x - mu)**2) / (2 * sigma**2))

# Verify it integrates to 1
result, _ = quad(normal_pdf, -np.inf, np.inf)
print(f"∫₋∞^∞ N(0,1) dx = {result:.6f}")

# Probability that X is between -1 and 1 (about 68%)
prob, _ = quad(normal_pdf, -1, 1)
print(f"P(-1 ≤ X ≤ 1) = {prob:.4f}")

# Using scipy.stats
print(f"Using scipy: P(-1 ≤ X ≤ 1) = {stats.norm.cdf(1) - stats.norm.cdf(-1):.4f}")

In [ ]:
# Visualize probability as area
x = np.linspace(-4, 4, 1000)
y = normal_pdf(x)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Standard normal
axes[0].plot(x, y, 'b-', linewidth=2)
x_fill = np.linspace(-1, 1, 100)
axes[0].fill_between(x_fill, normal_pdf(x_fill), alpha=0.3, color='blue')
axes[0].set_title(f'P(-1 ≤ X ≤ 1) = {prob:.2%}')
axes[0].set_xlabel('x')
axes[0].set_ylabel('Density')
axes[0].grid(True, alpha=0.3)

# Different probabilities
probs = [(1, 0.6827), (2, 0.9545), (3, 0.9973)]
axes[1].plot(x, y, 'b-', linewidth=2)
for sigma, prob in probs:
    axes[1].axvline(x=-sigma, color='gray', linestyle='--', alpha=0.5)
    axes[1].axvline(x=sigma, color='gray', linestyle='--', alpha=0.5)
axes[1].set_title('Standard Normal: 68-95-99.7 Rule')
axes[1].set_xlabel('x')
axes[1].set_ylabel('Density')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4.2 Cumulative Distribution Function

$$F(x) = P(X \leq x) = \int_{-\infty}^x f(t) \, dt$$

In [ ]:
# Compute and plot CDF
x = np.linspace(-4, 4, 1000)

# PDF
pdf = normal_pdf(x)

# CDF (numerical integration)
cdf = np.array([quad(normal_pdf, -np.inf, xi)[0] for xi in x])

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(x, pdf, 'b-', linewidth=2)
axes[0].set_title('Probability Density Function (PDF)')
axes[0].set_xlabel('x')
axes[0].set_ylabel('f(x)')
axes[0].grid(True, alpha=0.3)

axes[1].plot(x, cdf, 'r-', linewidth=2)
axes[1].axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
axes[1].axvline(x=0, color='gray', linestyle='--', alpha=0.5)
axes[1].set_title('Cumulative Distribution Function (CDF)')
axes[1].set_xlabel('x')
axes[1].set_ylabel('F(x) = P(X ≤ x)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4.3 Joint Probability Distributions

In [ ]:
# Joint normal distribution
def joint_normal(x, y, mu_x=0, mu_y=0, sigma_x=1, sigma_y=1, rho=0):
    """Bivariate normal PDF."""
    z = ((x - mu_x)**2 / sigma_x**2 - 
         2*rho*(x - mu_x)*(y - mu_y)/(sigma_x*sigma_y) + 
         (y - mu_y)**2 / sigma_y**2)
    return (1 / (2*np.pi*sigma_x*sigma_y*np.sqrt(1-rho**2))) * \
           np.exp(-z / (2*(1-rho**2)))

# Verify it integrates to 1
result, _ = dblquad(lambda y, x: joint_normal(x, y), -5, 5, lambda x: -5, lambda x: 5)
print(f"∫∫ joint_normal(x, y) dy dx = {result:.6f}")

In [ ]:
# Visualize joint distributions with different correlations
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

x = np.linspace(-3, 3, 100)
y = np.linspace(-3, 3, 100)
X, Y = np.meshgrid(x, y)

correlations = [0, 0.7, -0.7]
titles = ['Independent (ρ=0)', 'Positive correlation (ρ=0.7)', 'Negative correlation (ρ=-0.7)']

for ax, rho, title in zip(axes, correlations, titles):
    Z = joint_normal(X, Y, rho=rho)
    contour = ax.contourf(X, Y, Z, levels=20, cmap='viridis')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title(title)
    ax.set_aspect('equal')

plt.tight_layout()
plt.show()

---
# 5. EXPECTED VALUE AND MOMENTS

## 5.1 Expected Value

$$E[X] = \int_{-\infty}^{\infty} x \cdot f(x) \, dx$$

$$E[g(X)] = \int_{-\infty}^{\infty} g(x) \cdot f(x) \, dx$$

In [ ]:
# Expected value of standard normal
E_X, _ = quad(lambda x: x * normal_pdf(x), -np.inf, np.inf)
print(f"E[X] for N(0,1) = {E_X:.6f}")

# E[X²] for standard normal
E_X2, _ = quad(lambda x: x**2 * normal_pdf(x), -np.inf, np.inf)
print(f"E[X²] for N(0,1) = {E_X2:.6f}")

# Variance = E[X²] - E[X]²
Var_X = E_X2 - E_X**2
print(f"Var[X] = E[X²] - E[X]² = {Var_X:.6f}")

In [ ]:
# Expected value with different distribution: Exponential(λ=1)
def exp_pdf(x, lam=1):
    return lam * np.exp(-lam * x) if x >= 0 else 0

# E[X] for Exp(1) = 1/λ = 1
E_X, _ = quad(lambda x: x * exp_pdf(x), 0, np.inf)
print(f"\nE[X] for Exp(1) = {E_X:.6f} (analytical: 1)")

# E[X²] = 2/λ² = 2
E_X2, _ = quad(lambda x: x**2 * exp_pdf(x), 0, np.inf)
print(f"E[X²] for Exp(1) = {E_X2:.6f} (analytical: 2)")

# Var[X] = 1/λ² = 1
Var_X = E_X2 - E_X**2
print(f"Var[X] = {Var_X:.6f} (analytical: 1)")

## 5.2 Higher Moments

- **Skewness** (3rd moment): Measures asymmetry
- **Kurtosis** (4th moment): Measures tail heaviness

In [ ]:
# Compute moments for standard normal
def moment(n, pdf, lower=-np.inf, upper=np.inf):
    """Compute nth raw moment."""
    result, _ = quad(lambda x: x**n * pdf(x), lower, upper)
    return result

print("Raw moments of N(0,1):")
for n in range(1, 5):
    m = moment(n, normal_pdf)
    print(f"E[X^{n}] = {m:.6f}")

# Note: Odd moments are 0 for symmetric distributions

## 5.3 Moment Generating Functions

$$M_X(t) = E[e^{tX}] = \int_{-\infty}^{\infty} e^{tx} f(x) \, dx$$

In [ ]:
# MGF for normal distribution: M(t) = e^(μt + σ²t²/2)
# For N(0,1): M(t) = e^(t²/2)

def mgf_numerical(t, pdf, lower=-10, upper=10):
    """Compute MGF numerically."""
    result, _ = quad(lambda x: np.exp(t*x) * pdf(x), lower, upper)
    return result

def mgf_normal(t):
    """Analytical MGF for N(0,1)."""
    return np.exp(t**2 / 2)

print("MGF of N(0,1):")
for t in [0, 0.5, 1.0, 1.5]:
    numerical = mgf_numerical(t, normal_pdf)
    analytical = mgf_normal(t)
    print(f"M({t}) = {numerical:.4f} (analytical: {analytical:.4f})")

---
# 6. MULTIVARIATE CALCULUS

## 6.1 Gradient and Directional Derivatives

The gradient is a vector of partial derivatives:

$$\nabla f = \left( \frac{\partial f}{\partial x}, \frac{\partial f}{\partial y} \right)$$

In [ ]:
# Example: f(x, y) = x² + y²
# ∇f = (2x, 2y)

def f(x, y):
    return x**2 + y**2

def grad_f(x, y):
    return np.array([2*x, 2*y])

# Visualize gradient field
x = np.linspace(-2, 2, 20)
y = np.linspace(-2, 2, 20)
X, Y = np.meshgrid(x, y)
Z = f(X, Y)

# Compute gradients
U = 2 * X
V = 2 * Y

fig, ax = plt.subplots(figsize=(10, 8))

# Contour plot
contour = ax.contourf(X, Y, Z, levels=20, cmap='viridis', alpha=0.7)
plt.colorbar(contour, label='f(x,y) = x² + y²')

# Gradient vectors (arrows)
ax.quiver(X, Y, U, V, color='red', alpha=0.8)

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Gradient Field of f(x,y) = x² + y²\nArrows point in direction of steepest ascent')
ax.set_aspect('equal')

plt.show()

## 6.2 Line Integrals

Integral of a function along a curve.

In [ ]:
# Line integral of f(x,y) along a path
# Path: circle of radius 1
# x = cos(t), y = sin(t), 0 ≤ t ≤ 2π

def line_integral(f, x_t, y_t, dx_dt, dy_dt, t_range):
    """Compute line integral ∫f ds."""
    integrand = lambda t: f(x_t(t), y_t(t)) * np.sqrt(dx_dt(t)**2 + dy_dt(t)**2)
    result, _ = quad(integrand, t_range[0], t_range[1])
    return result

# Path: unit circle
x_t = lambda t: np.cos(t)
y_t = lambda t: np.sin(t)
dx_dt = lambda t: -np.sin(t)
dy_dt = lambda t: np.cos(t)

# f(x, y) = 1 (arc length)
f = lambda x, y: 1
arc_length = line_integral(f, x_t, y_t, dx_dt, dy_dt, [0, 2*np.pi])
print(f"Arc length of unit circle: {arc_length:.4f} (expected: 2π = {2*np.pi:.4f})")

# f(x, y) = x² + y²
f = lambda x, y: x**2 + y**2
result = line_integral(f, x_t, y_t, dx_dt, dy_dt, [0, 2*np.pi])
print(f"∫(x² + y²) ds over unit circle: {result:.4f} (expected: 2π = {2*np.pi:.4f})")

## 6.3 Divergence Theorem (Preview)

The divergence theorem relates surface integrals to volume integrals:

$$\iint_S \mathbf{F} \cdot d\mathbf{S} = \iiint_V \nabla \cdot \mathbf{F} \, dV$$

In [ ]:
# Simple example: F = (x, y, z) over unit sphere
# div F = ∂x/∂x + ∂y/∂y + ∂z/∂z = 3
# Volume integral: ∫∫∫ 3 dV = 3 × (4π/3) = 4π

# Numerical verification using Monte Carlo
n_samples = 100000
samples = np.random.uniform(-1, 1, (n_samples, 3))
# Keep only points inside unit sphere
inside = np.sum(samples**2, axis=1) <= 1
volume = 8 * np.sum(inside) / n_samples  # 8 = volume of [-1,1]³

# div F = 3
integral = 3 * volume

print(f"Volume of unit sphere (Monte Carlo): {volume:.4f} (exact: {4*np.pi/3:.4f})")
print(f"∫∫∫ div F dV = {integral:.4f} (exact: 4π = {4*np.pi:.4f})")

---
# 7. VISUALIZATIONS

## 7.1 3D Visualization of Double Integrals

In [ ]:
# Create 3D visualization
fig = plt.figure(figsize=(15, 5))

# Surface: z = sin(x) * cos(y)
x = np.linspace(0, np.pi, 50)
y = np.linspace(0, np.pi, 50)
X, Y = np.meshgrid(x, y)
Z = np.sin(X) * np.cos(Y)

# 3D surface plot
ax1 = fig.add_subplot(131, projection='3d')
ax1.plot_surface(X, Y, Z, cmap='coolwarm', alpha=0.8)
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('z')
ax1.set_title('z = sin(x)cos(y)')

# Contour plot
ax2 = fig.add_subplot(132)
contour = ax2.contourf(X, Y, Z, levels=20, cmap='coolwarm')
plt.colorbar(contour, ax=ax2)
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_title('Contour Plot')

# Heat map
ax3 = fig.add_subplot(133)
im = ax3.imshow(Z, extent=[0, np.pi, 0, np.pi], origin='lower', 
                cmap='coolwarm', aspect='auto')
plt.colorbar(im, ax=ax3)
ax3.set_xlabel('x')
ax3.set_ylabel('y')
ax3.set_title('Heat Map')

plt.tight_layout()
plt.show()

# Compute the integral
result, _ = dblquad(lambda y, x: np.sin(x) * np.cos(y), 0, np.pi, 
                    lambda x: 0, lambda x: np.pi)
print(f"∫₀^π ∫₀^π sin(x)cos(y) dy dx = {result:.4f}")

---
# 8. PRACTICE PROBLEMS

## Problem 1: Compute Expected Value

In [ ]:
# Problem: Find E[X] for f(x) = 2x on [0, 1]
# E[X] = ∫₀¹ x·2x dx = 2∫₀¹ x² dx = 2[x³/3]₀¹ = 2/3

def pdf(x):
    return 2*x if 0 <= x <= 1 else 0

# Verify it's a valid PDF
total, _ = quad(pdf, 0, 1)
print(f"∫ pdf = {total:.4f} (should be 1)")

# Compute E[X]
E_X, _ = quad(lambda x: x * pdf(x), 0, 1)
print(f"E[X] = {E_X:.4f} (analytical: 2/3 = {2/3:.4f})")

# Compute Var[X]
E_X2, _ = quad(lambda x: x**2 * pdf(x), 0, 1)
Var_X = E_X2 - E_X**2
print(f"Var[X] = {Var_X:.4f} (analytical: 1/18 = {1/18:.4f})")

## Problem 2: Double Integral Over Triangle

In [ ]:
# Problem: ∫∫ (x + y) dA over triangle with vertices (0,0), (1,0), (0,1)
# Region: 0 ≤ x ≤ 1, 0 ≤ y ≤ 1-x
# = ∫₀¹ ∫₀^(1-x) (x+y) dy dx

def f(y, x):
    return x + y

result, _ = dblquad(f, 0, 1, lambda x: 0, lambda x: 1-x)

print("∫∫ (x+y) dA over triangle")
print(f"Numerical: {result:.6f}")
print(f"Analytical: 1/3 = {1/3:.6f}")

## Problem 3: Monte Carlo for High-Dimensional Integral

In [ ]:
# Problem: Volume of 5-dimensional unit ball
# V_n = π^(n/2) / Γ(n/2 + 1)
# For n=5: V_5 = π^2.5 / Γ(3.5) = 8π²/15

from scipy.special import gamma

n_dim = 5
n_samples = 1000000

# Monte Carlo
samples = np.random.uniform(-1, 1, (n_samples, n_dim))
inside = np.sum(samples**2, axis=1) <= 1
volume_mc = 2**n_dim * np.sum(inside) / n_samples

# Analytical
volume_exact = np.pi**(n_dim/2) / gamma(n_dim/2 + 1)

print(f"Volume of {n_dim}-dimensional unit ball")
print(f"Monte Carlo: {volume_mc:.4f}")
print(f"Analytical: {volume_exact:.4f}")
print(f"Error: {abs(volume_mc - volume_exact):.4f}")

---
## Summary

### Key Concepts:

1. **Definite Integrals**: Area under curve, accumulated quantities
2. **Numerical Integration**: Riemann sums, trapezoidal, Simpson's, Monte Carlo
3. **Multiple Integrals**: Double/triple integrals, change of variables
4. **Probability**: PDFs, CDFs, expected values from integrals
5. **Moments**: Mean, variance, skewness, kurtosis
6. **Multivariate Calculus**: Gradients, line integrals

### ML Applications:

- **Maximum Likelihood**: Integrating over parameter space
- **Bayesian Inference**: Computing posterior distributions
- **Expected Value**: Loss functions, risk
- **Monte Carlo Methods**: Sampling-based inference
- **Normalizing Flows**: Computing partition functions

### Important Formulas:

$$E[X] = \int x f(x) dx$$
$$Var[X] = E[X^2] - E[X]^2$$
$$P(a \leq X \leq b) = \int_a^b f(x) dx$$

---

**Next Steps:**
1. Practice computing integrals by hand
2. Study numerical methods in depth
3. Apply to probabilistic machine learning
4. Learn about variational inference